In [10]:
import torch
import torch.nn as nn
import math

In [ ]:
class LSTMCell(nn.Module):
  """
  seq_length : 332일치의 데이터가 있을 때 seq_length가 10이라면 10일치의 데이터가 하나의 시퀀스임. 그리고 시퀀스 개수는 332-10+1=323개.

  batch_size : 여러 개의 시퀀스를 한 번에 처리하기 위한 묶음. 332일치의 데이터가 있을 때 batch_size가 32라면 10개의 시퀀스가 32개(연속된
               320일 치의 데이터가 있다는 게 아님). 그러므로 이 경우 배치 하나의 shape은 (32,10,2).

  hidden_size : hidden_state의 dimension(shape OR feature 개수). hidden_state의 dimension은 input_data의 dimension(feature)과 독립.
                hidden_state는 과거의 모든 입력 정보와 LSTM의 내부 계산을 반영하여 다음 타임스텝으로 전달하는 역할이기 때문. 즉,
                LSTM이 input_data를 처리한 결과로 생성된 학습 표현(learned representation).

  hidden_state의 shape은 (num_layers, batch_size, hidden_size), input_data의 shape은 (batch_size, seq_length, feature_size).
  cell_state의 shape은 (num_layers, batch_size, hidden_size), output의 shape은 (batch_size, seq_length, hidden_size). 이때 각 LSTM Cell에 들어가는
  Xt의 shape은 (1,1,feature_size).
  => 가중치 행렬은 이의 shape을 맞춰줘야 함.

  nn.Module은 파이토치에서 모든 nueral network 구성에 필수. 미분과 매개변수 관리 등을 자동으로 해줌

  nn.Module을 상속하게 되면 기본적으로 __init__()과 foward()를 override하게 됨

  파라미터 reset 필요.

  """

  # 모델의 변수를 초기화하는 매서드
  def __init__(self, input_size, seq_length, hidden_size, bias=True):
    super(LSTMCell, self).__init__()
    # 각 Cell에 들어올 input data feature(차원, dimension) 개수(크기)
    self.input_size = input_size
    # 각 Cell에 들어올 input data 개수
    self.seq_length = seq_length
    # hidden state의 feature(차원, dimension) 개수(크기)
    self.hidden_size = hidden_size
    # 편향
    self.bias = bias

    # 학습에 필요한 input gate 파라미터들을 생성
    self.Wx_i = torch.Parameters(torch.Tensor((self.input_size, self.hidden_size))) # x에 대한 가중치 탠서
    self.Wh_i = torch.Parameters(torch.Tensor((self.hidden_size, self.hidden_size))) # h에 대한 가중치 탠서
    self.b_i = torch.Parameters(torch.Tensor((self.hidden_size, self.hidden_size)))
    # 학습에 필요한 forget gate 파라미터들을 생성
    self.Wx_f = torch.Parameters(torch.Tensor((self.input_size, self.hidden_size)))
    self.Wh_f = torch.Parameters(torch.Tensor((self.hidden_size, self.hidden_size)))
    self.b_f = torch.Parameters(torch.Tensor((self.hidden_size, self.hidden_size)))
    # 학습에 필요한 input gate 파라미터들을 생성
    self.Wx_o = torch.Parameters(torch.Tensor((self.input_size, self.hidden_size)))
    self.Wh_o = torch.Parameters(torch.Tensor((self.hidden_size, self.hidden_size)))
    self.b_o = torch.Parameters(torch.Tensor((self.hidden_size, self.hidden_size)))
    # 학습에 필요한 예비 cell_state 파라미터들을 생성
    self.Wx_c = torch.Parameters(torch.Tensor((self.input_size, self.hidden_size)))
    self.Wh_c = torch.Parameters(torch.Tensor((self.hidden_size, self.hidden_size)))
    self.b_c = torch.Parameters(torch.Tensor((self.hidden_size, self.hidden_size)))

  # input data에 모델이 적용할 연산을 구현하는 매서드
  def forward(self,x,hidden):
    # hidden 변수에 이전 시점의 hidden_state, cell_state가 들어있음. set형태로.
    h_prev, c_prev = hidden

    # Input gate
    i_t = torch.sigmoid(x @ self.Wx_i + h_prev @ self.Wh_i + self.b_i)

    # Forget gate
    f_t = torch.sigmoid(x @ self.Wx_f + h_prev @ self.Wh_f + self.b_f)

    # 예비 cell_state값
    c_tilde = torch.tanh(x @ self.Wx_c + h_prev @ self.Wh_c + self.b_c)

    # Output gate
    o_t = torch.sigmoid(x @ self.Wx_o + h_prev @ self.Wh_o + self.b_o)

    # Cell state
    c_t = f_t * c_prev + i_t * c_tilde

    # hidden state
    h_t = o_t * torch.tanh(c_t)

    return h_t, c_t

class LSTM(nn.Module):
  def __init__(self, num_layers):
    self.num_layers = num_layers

In [8]:
input_size = 2
seq_length = 10
hidden_size = 100
batch_size = 8
x = torch.randn([1,1,input_size])
h_t = torch.randn([1, batch_size, hidden_size])
Wx_i = torch.randn([input_size, hidden_size])
Wh_i = torch.randn([hidden_size, hidden_size])

In [9]:
torch.sigmoid(x @ Wx_i + h_t @Wh_i)

tensor([[[9.9981e-01, 1.2498e-01, 2.8881e-02, 9.9973e-01, 9.9999e-01,
          1.0000e+00, 3.7503e-04, 9.6060e-02, 8.7857e-04, 1.0000e+00,
          1.1323e-03, 1.2335e-04, 4.8179e-04, 1.6290e-03, 9.9990e-01,
          9.2669e-01, 9.9997e-01, 9.9990e-01, 6.3876e-09, 9.9363e-01,
          1.2494e-01, 5.6763e-03, 1.0337e-03, 5.0390e-06, 8.7731e-01,
          6.0464e-01, 3.2510e-04, 3.9635e-05, 9.8400e-01, 9.6908e-01,
          6.8506e-04, 1.2726e-02, 2.8434e-04, 3.3788e-09, 1.6714e-11,
          9.9987e-01, 2.8926e-08, 2.6368e-04, 9.8485e-01, 1.0000e+00,
          9.9392e-05, 5.8612e-04, 1.0000e+00, 8.7897e-09, 9.9815e-01,
          1.0000e+00, 9.9624e-01, 9.8852e-01, 9.9996e-01, 9.9995e-01,
          9.9823e-01, 9.7229e-01, 9.9979e-01, 9.9947e-01, 2.9737e-05,
          4.0152e-05, 4.9982e-01, 9.3108e-01, 1.0000e+00, 5.1607e-06,
          3.1855e-03, 9.9794e-01, 2.4156e-05, 9.4130e-01, 9.9999e-01,
          8.9068e-03, 9.0774e-01, 9.2075e-01, 3.9013e-03, 9.3578e-07,
          2.5513e-04

In [ ]:
# 데이터는 332X2임. feature는 가격과 날짜. 총 332일치의 나스닥지수 데이터